#Loading Necessary Libraries



In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
plt.rcParams['figure.figsize'] = (12, 8)
import seaborn as sns
sns.set(style='whitegrid', color_codes=True)
from sklearn.metrics import confusion_matrix

# Preparing the data

Import the cleaned dataset

In [ ]:
df=pd.read_excel("/content/PiCOS_cleaned_data.xlsx")
df.head()

,PCOS (Y/N),Age (yrs),Weight (Kg),Height(Cm),BMI,Blood Group,Cycle(R/I),Cycle length(days),Marraige Status (Yrs),Pregnant(Y/N),...,Hip(inch),Waist(inch),Waist:Hip Ratio,Weight gain(Y/N),hair growth(Y/N),Skin darkening (Y/N),Hair loss(Y/N),Pimples(Y/N),Fast food (Y/N),Reg.Exercise(Y/N)
0,0,28,44.6,152.0,19.300000,15,2,5,7.0,0,...,36,30,0.833333,0,0,0,0,0,1,0
1,0,36,65.0,161.5,24.921163,15,2,5,11.0,1,...,38,32,0.842105,0,0,0,0,0,0,0
2,1,33,68.8,165.0,25.270891,11,2,5,10.0,1,...,40,36,0.900000,0,0,0,1,1,1,0
3,0,37,65.0,148.0,29.674945,13,2,5,4.0,0,...,42,36,0.857143,0,0,0,0,0,0,0
4,0,25,52.0,161.0,20.060954,11,2,5,1.0,1,...,37,30,0.810811,0,0,0,1,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PCOS (Y/N)             541 non-null    int64  
 1   Age (yrs)              541 non-null    int64  
 2   Weight (Kg)            541 non-null    float64
 3   Height(Cm)             541 non-null    float64
 4   BMI                    541 non-null    float64
 5   Blood Group            541 non-null    int64  
 6   Cycle(R/I)             541 non-null    int64  
 7   Cycle length(days)     541 non-null    int64  
 8   Marraige Status (Yrs)  541 non-null    float64
 9   Pregnant(Y/N)          541 non-null    int64  
 10  No. of aborptions      541 non-null    int64  
 11  Hip(inch)              541 non-null    int64  
 12  Waist(inch)            541 non-null    int64  
 13  Waist:Hip Ratio        541 non-null    float64
 14  Weight gain(Y/N)       541 non-null    int64  
 15  hair g

Load the cleaned dataset and perform any required preprocessing steps, since the data has been cleaned neatly by dropping the unnecessary columns and removing any nulls, we can proceed to the next step

We label the data to fit the neural networks. The X should contains PCOS symptoms so we drop the targeted label (y) and set the axis to 1 (this is to make sure it drops the columns

Then, we set the targeted label (y) to whether someone is diagnosed with PCOS or not

In [ ]:
X  = df.drop('PCOS (Y/N)', axis = 1)
y = df[["PCOS (Y/N)"]]

In [ ]:
X.shape

(541, 20)

In [ ]:
y.shape

(541, 1)

After labeling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify= y)

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.2, random_state=0, stratify= y_train)
X_train.shape

(388, 20)

In [ ]:
X_test.shape

(55, 20)

In [ ]:
X_dev.shape

(98, 20)

In [ ]:
X_train.shape

(388, 20)

In [ ]:
y_train.shape

(388, 1)

In [ ]:
y_train.shape

(388, 1)

In [ ]:
scaler = MinMaxScaler().fit(X_train)


X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train)

X_dev = scaler.transform(X_dev)
X_dev = pd.DataFrame(X_dev)

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test)


# Setting Column Names from dataset
X_train.columns = X.columns
X_test.columns = X.columns
X_dev.columns = X.columns

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import datasets, layers, models

In [ ]:
X_train = X_train.to_numpy().reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)
X_dev = X_dev.to_numpy().reshape(X_dev.shape[0], X_dev.shape[1], 1)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
y_dev = to_categorical(y_dev, 2)

In [ ]:
# Define the number of input features
input_shape = X_train.shape[1:]

# Build the Sequential model
model = Sequential()
model.add(layers.Conv1D(64, (3), activation='relu', input_shape=input_shape))
model.add(layers.Conv1D(64, (3), activation='relu'))
model.add(layers.Flatten())
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dense(2, activation='sigmoid'))

# Compile the model
learning_rate = 0.0005
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

model.build(X_train.shape[1:])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 64)            256       
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 64)            12352     
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 2)                 3

In [ ]:
epochs = 100
batch_size = 12

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1,validation_data= (X_dev,y_dev))

Epoch 1/100
33/33 [==============================] - 2s 14ms/step - loss: 0.7085 - accuracy: 0.6675 - val_loss: 0.6546 - val_accuracy: 0.6735
Epoch 2/100
33/33 [==============================] - 0s 6ms/step - loss: 0.5999 - accuracy: 0.6727 - val_loss: 0.5651 - val_accuracy: 0.6735
Epoch 3/100
33/33 [==============================] - 0s 6ms/step - loss: 0.5265 - accuracy: 0.6727 - val_loss: 0.5334 - val_accuracy: 0.6735
Epoch 4/100
33/33 [==============================] - 0s 5ms/step - loss: 0.5042 - accuracy: 0.6753 - val_loss: 0.5321 - val_accuracy: 0.6735
Epoch 5/100
33/33 [==============================] - 0s 6ms/step - loss: 0.4910 - accuracy: 0.7036 - val_loss: 0.5250 - val_accuracy: 0.8367
Epoch 6/100
33/33 [==============================] - 0s 7ms/step - loss: 0.4787 - accuracy: 0.8093 - val_loss: 0.5232 - val_accuracy: 0.8163
Epoch 7/100
33/33 [==============================] - 0s 5ms/step - loss: 0.4712 - accuracy: 0.8479 - val_loss: 0.5163 - val_accuracy: 0.8367
Epoch 8/100


In [ ]:
print(y_train.shape)

(388, 2)


In [ ]:
pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model.save('model.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('model.h5')